In [100]:
### Import das bibliotecas necessarias
import pandas as pd
from sqlalchemy import create_engine

In [101]:
try:
	# Criando a string de conexão
    user = "root"
    password = "admin123"
    host = "localhost"
    database = "dbprojetoimpacta"
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"

    # Crie a conexão com o banco de dados usando SQLAlchemy
    engine = create_engine(connection_string)
    print('Conectado ao banco de dados...')

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Conectado ao banco de dados...


In [102]:
# API banco central cotação dolar
path = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2019'&@dataFinalCotacao='12-31-2025'&$top=9000&$format=text/csv&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
# Leitura dos dados e criação do dataframe
tabela = pd.read_csv(path)
# Renomeando as colunas do dataframe
tabela = tabela.rename(columns={'cotacaoCompra': 'Cotacao de Compra','cotacaoVenda':'Cotacao de Venda','dataHoraCotacao':'Data e Hora da Cotacao'})
# Separando a tabela data e hora em duas tabelas
tabela[['Data', 'Hora']] = tabela['Data e Hora da Cotacao'].str.split(' ', 1, expand=True)
# Apagando a coluna com data e hora juntos
tabela.drop(columns=['Data e Hora da Cotacao'], inplace=True)

In [103]:
# Converter os tipos e organizar os dados das colunas 

tabela['Cotacao de Compra'] = tabela['Cotacao de Compra'].astype(str).str.replace(',', '.').astype(float)
tabela['Cotacao de Venda'] = tabela['Cotacao de Venda'].astype(str).str.replace(',', '.').astype(float)
tabela['Data'] = pd.to_datetime(tabela['Data'], format='%Y-%m-%d')
tabela['Hora'] = pd.to_datetime(tabela['Hora'], format='%H:%M:%S.%f').dt.time
tabela['Hora'] = pd.to_datetime(tabela['Hora'], format='%H:%M:%S.%f', errors='coerce')
print(tabela[tabela['Hora'].isna()])


Empty DataFrame
Columns: [Cotacao de Compra, Cotacao de Venda, Data, Hora]
Index: []


In [104]:
print(tabela.dtypes)

Cotacao de Compra           float64
Cotacao de Venda            float64
Data                 datetime64[ns]
Hora                 datetime64[ns]
dtype: object


In [105]:
try:
    # Salvando o DataFrame no banco de dados MySQL
    tabela.to_sql(name='cotacoes', con=engine, if_exists='replace', index=False)
    print('Dados gravados no banco de dados...')


except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados gravados no banco de dados...


In [108]:
# Executa a consulta SQL para contar as linhas da tabela
result = engine.execute('SELECT COUNT(*) FROM cotacoes')
count = result.fetchone()[0]

print(f'O número de linhas na tabela é: {count}')

O número de linhas na tabela é: 1070


In [109]:
# Fechando a conexão com o banco de dados
engine.dispose()